In [39]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import os as os
import pandas as pd
import re
keywordlist=["health","education"]
max_num=5
datasets_array=[]
dataset_tag_dict={}
for keyword in keywordlist:
    temp_dict_for_datasets_and_tag={}
    count=0
    link_list=[]
    first_url="https://data.cityofnewyork.us/browse?amp=&q="+keyword+"&sortBy=relevance&page=1"
    link_list.append(first_url)
    paginationlink = requests.get(first_url)
    soup = BeautifulSoup(paginationlink.text, "html.parser")
    links=soup.findAll("a",{"class":"pageLink"})
    for link in links:
        link_list.append("https://data.cityofnewyork.us"+link['href'])
    print(link_list)
    for url in link_list:
        if(count<max_num):
            datalink1="https://dev.socrata.com/foundry/data.cityofnewyork.us/"
            datalink2='https://data.cityofnewyork.us/resource/'
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            # parent div with meta tag
            links=soup.findAll("div", {"class": "browse2-result"})
            dict2={}
            #print("url in link")
        if(count<max_num):
            count2=0
            for link in links:
                tags_array=[]
                if(count2<max_num):
                    try:
                        # get href and add .json at the end
                        href2=link.findAll("a",{"class":"browse2-result-api-link"})[0]["href"].replace(datalink1,datalink2)
                        href=href2+".json"
                        # Get the tags from here
                        #print("link is : ",href2)
                        title=link.findAll("a",{"class":"browse2-result-name-link"})[0].string
                        dict2[title]=href
                        taglist=link.findAll("div",{"class":"browse2-result-topics"})
                        tag_arr=taglist[0].findAll("a")
                        for tag in tag_arr:
                            tags_array.append(tag.find("span").string)
                        temp_dict_for_datasets_and_tag[title]=tags_array
                        count2=count2+1
                    except IndexError:
                        pass
                        #print(link.findAll("a",{"class":"browse2-result-name-link"})[0].string)
            #print(dict2)
        for key in dict2:
            if(count<max_num):
                try:
                    data=pd.read_json(dict2[key])
                    datasets_array.append(re.sub("/","",key)+".csv")
                    #print(key)
                    if not (os.path.exists("upload/"+re.sub("/","",key)+".csv")):
                        data.to_csv(r"upload/"+re.sub("/","",key)+".csv",sep=",")
                        count=count+1
                        print("fetched: ",count,"\n")
                except:
                    print("error in file location")
    dataset_tag_dict[keyword]=temp_dict_for_datasets_and_tag

['https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=2', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=3', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=4', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=5', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=6', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=7', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=8', 'https://data.cityofnewyork.us/browse?amp=&q=health&sortBy=relevance&page=9']
fetched:  1 

fetched:  2 

fetched:  3 

fetched:  4 

fetched:  5 

['https://data.cityofnewyork.us/browse?amp=&q=education&sortBy=relevance&page=1', 'https://data.cityofnewyork.us/browse?amp=&q=education&sortBy=relev

In [45]:
for key in dataset_tag_dict:
    print(key,"----------" ,dataset_tag_dict[key])

health ---------- {'Community Health Survey': ['dohmh', 'health', 'survey'], 'DOHMH Community Health Survey (2010-2016)': ['new york city adults', 'surveys', 'public health'], 'NYC Health + Hospitals patient care locations - 2011': ['healthy living', 'medicaid', 'medicare', 'hospital', 'healthcare provider'], 'Community Health Centers': ['health', 'community', 'hra', 'human resources administration', 'jobs and economic mobility'], 'NYC Health + Hospitals WTC 9/11 Health Center – common disorders – 2011': ['healthy living', 'environmental health center', 'disorder', 'wtc', 'world trade center'], 'DOHMH Community Mental Health Survey': ['serious mental illness (smi)', 'new york city adults', 'surveys', 'mental health']}
education ---------- {'2017 – 2018 Anticipated Bilingual Education Programs': ['dual language', 'transitional', 'bilingual', 'doe'], 'TLC Driver Education 24 Hour Course Providers (Dataset)': ['course', 'education', 'driver', 'tlc'], 'Office Of Adult And Continuing Educat